In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool 
import time
from functools import partial 
import os
%matplotlib inline

DATA_DIR = "../gamedata/original_dataset/"
DATA_FILE = "dd_event_schemas.csv"
GenreStats_DIR = "../gamedata/genreStats/"
PlatformStats_DIR = "../gamedata/platformStats/"
GenreScore_DIR = "../gamedata/genreScore/"
PlatformScore_DIR = "../gamedata/platformScore/"
WeightedScore_DIR = "../gamedata/weightedScore/"
TrainDataset_DIR = "../gamedata/train_dataset/"

dataset=pd.read_csv(DATA_DIR+DATA_FILE)#load original dataset

#Select a sub dataset where the genre equals to 'No genre' 
subdataset = dataset[dataset['genre']=='No genre']
#List of all events in 'No genre' subdataset
events = subdataset['eventname'].drop_duplicates()
#Count the amount of all event
#events_amount=events.count()
#List of all gameid  in 'No genre' subdataset
games = subdataset['gameid'].drop_duplicates()

events=list(set(events))
events.sort()

#get all gameid in this genre
def get_all_gameid(dataset):
    AllGameId = dataset['gameid']
    AllGameId = AllGameId.drop_duplicates()
    
    return AllGameId

#get all events in this game
def get_all_gameEvents(gameId,dataset):
    findGame = dataset[dataset['gameid'] == gameId]
    eventsInGame = findGame.drop_duplicates(['eventname'])
    eventsList = eventsInGame['eventname']
    
    return eventsList

allGameId = get_all_gameid(subdataset)

def get_feature_vector(gameId,eventsVocabulary,dataset):
    ''' Map the presence of events to a vector
    :param gameId
    :type gameId: list of int
    :param eventsVocabulary 
    :type eventsVocabulary : list of str
    :param dataset
    :type dataset: dataframe
    :return feature_vect
    :type feature_vect: list of int
    '''
    eventsList = get_all_gameEvents(gameId,dataset)
    feature_vect = [0]*len(eventsVocabulary)
    
    for event in eventsList:
        if event in eventsVocabulary:
            idx = eventsVocabulary.index(event)
            #print(idx)
            feature_vect[idx] = 1
    return feature_vect      

train_dataset = [get_feature_vector(gameId,events,subdataset) for gameId in allGameId]

train_dataset = pd.DataFrame(train_dataset)
type(train_dataset)

#amount of each event
num_events = np.sum(train_dataset,axis=0)

#Calculate P(ei|ej)
def calculateProb(i):
    eiej_tmp = pd.DataFrame()
    arr1=[]
    arr2=[]
    #amount = num 
    #for i in amount:
    arr1=[]
    for j in range(100):
        eiej_tmp['i'] = train_dataset.iloc[:][i]
        eiej_tmp['j'] = train_dataset.iloc[:][j]
        eiej = eiej_tmp[(eiej_tmp['i']==1) & (eiej_tmp['j']==1)]
        p = len(eiej)/np.sum(eiej_tmp['i'])
        #print('p(e'+str(j)+'|e'+str(i)+ ')='+str(p) )
        arr1.append(p)
    #print(arr1)
    #arr2.append(arr1)
    print(arr1)
    return arr1

start = time.clock()
amount = range(len(events))
pool = ThreadPool(16) 
matrix = pool.map(calculateProb, amount)
pool.close()
pool.join()
end = time.clock()

#matrix = matrix = pd.DataFrame(calculateProb(train_dataset))

print(end-start)
newmatrix = pd.DataFrame(matrix)

newmatrix.to_csv('probMatrix.csv')

In [ ]:
def get_newgame_feature(eventsList,eventsVocabulary):
    ''' Map the presence of events to a vector
    :param eventsList 
    :type list of events
    :param eventsVocabulary 
    :type eventsVocabulary : list of str
    :return feature_vect
    :type feature_vect: list of int
    '''
    feature_vect = [0]*len(eventsVocabulary)
    
    for event in eventsList:
        if event in eventsVocabulary:
            idx = eventsVocabulary.index(event)
#             print(idx)
            feature_vect[idx] = 1
    return feature_vect 